In [ ]:
import os
import pandas as pd
import sys
base_path = os.getcwd()
sys.path.append(os.path.abspath(os.path.join(base_path, '../src')))
from utils import importer as imp
from utils import helper as hp
from utils import exporter as ex
import plotly.graph_objects as go
import plotly.express as px
import re
import seaborn as sns
import matplotlib.pyplot as plt
from IPython.display import display, HTML
import openpyxl

In [ ]:
parent_dir = os.path.dirname(os.getcwd())
os.chdir(parent_dir)

In [ ]:
def display_table_with_caption(df, caption, table_number):
    html = f"""
    {df.to_html(index=False)}
    <p style="font-style: italic; font-size: 0.9em; color: gray; margin-top: 5px;">
        Table {table_number}: {caption}
    </p>
    """
    display(HTML(html))
    
def add_figure_caption(fig, figure_number, caption_text):
    full_caption = f"Figure {figure_number}: {caption_text}"
    fig.text(0.01, 0.01, full_caption, ha='left', fontsize=10, style='italic', color='gray')
    plt.tight_layout(rect=[0, 0.03, 1, 1])
    
def plotly_color_to_matplotlib(c):

    if c.startswith('rgb'):
        nums = list(map(float, re.findall(r"[\d\.]+", c)))
        if len(nums) == 3:
            r, g, b = [int(x) for x in nums]
            return '#{:02x}{:02x}{:02x}'.format(r, g, b)
        elif len(nums) == 4:
            r, g, b, a = nums
            return (r/255, g/255, b/255, a)

    return c

In [ ]:
# [1] db export
sql_path1 = "src/sql/mapping_overview.sql"
sql_path2 = "src/sql/episodic_modelling.sql"

map_df =  ex.db_export(sql_query=sql_path1)
episode = ex.db_export(sql_query=sql_path2)
sql = "Select e.episode_id, c.concept_name, ee.event_id from cdm.episode e inner join cdm.episode_event ee on e.episode_id = ee.episode_id left join cdm.concept c on e.episode_concept_id = c.concept_id "
episode_event = ex.execute_sql(sql)

# [2] source import
imp_dataframes = imp.source_import() 
for name, df in imp_dataframes.items():
    globals()[name] = df
patient = imp_dataframes['patient']
tumor = imp_dataframes['tumor']
#map_df.to_csv('map_df.csv', index=False)

# [3] appendix
agg_df = map_df.groupby([
    'table_name',             # CDM table
    'vocabulary_id',          # Target vocabulary
    'source_vocabulary_id',   # Source vocabulary
    'concept_class_id'        # Concept class
]).agg(
    distinct_count=('source_value', pd.Series.nunique),
    count=('record_count', 'sum')
).reset_index()

# total number per domain for percentage calculation
table_totals = map_df.groupby('table_name')['record_count'].sum().rename("table_total")
total_sum = map_df['record_count'].sum()


# merge table_totals in agg_df (on table_name)
agg_df = agg_df.merge(table_totals, on='table_name', how='left')


# calculate percentage
agg_df['percent [%] per CDM table'] = (agg_df['count'] / agg_df['table_total'] * 100).round(2)
agg_df['percent [%] total'] = (agg_df['count'] / total_sum * 100).round(2)

agg_df = agg_df.drop(columns=['table_total'])

# rename columns
agg_df = agg_df.rename(columns={
    'table_name': 'CDM table',
    'vocabulary_id': 'Target vocabulary',
    'source_vocabulary_id': 'Source vocabulary',
    'concept_class_id': 'Concept class',
    'distinct_count': 'Distinct count',
    'count': 'Count',
})

print(agg_df)
agg_df.to_excel('appendix_mapping_overview.xlsx', index=False)


In [ ]:
# Mapping Overview Analysis

# [1a] overall patient transmission rate
patient_count = patient['Geschlecht'].value_counts().reset_index()
patient_count['source'] = 'zfkd'
patient_count = patient_count.rename(columns={'Geschlecht': 'concept_name', 'count': 'record_count'})
gender_map = {
    'W': 'FEMALE',
    'M': 'MALE',
    'D': 'DIVERSE',
    'U': 'UNKNOWN',
    'X': 'UNSPECIFIED'  
}
patient_count['concept_name'] = patient_count['concept_name'].map(gender_map)
    # cdm
person_count = map_df[map_df['table_name'] == 'person'][['concept_name', 'record_count']]
person_count['source'] = 'cdm'
    # pivotierung
trans_rate = pd.concat([patient_count, person_count], ignore_index=True)
pivot_trans = trans_rate.pivot_table(
    index='concept_name',
    columns='source',
    values='record_count',
    fill_value=0
).reset_index()
pivot_trans['percent [%]'] = (pivot_trans['cdm'] / pivot_trans['zfkd'] * 100).round(2)


# [1b] overall tumor transmission rate
    # icdo3 analysis
total_tumors = tumor.shape[0]
icdo3 = map_df[
    (map_df['table_name'] == 'condition_occurrence') &
    (map_df['concept_class_id'] == 'ICDO Condition')
]
total_icdo3 = icdo3['record_count'].sum()
total_patients = patient.shape[0]
icdo3_per_tumor_percentage = (total_icdo3 / total_tumors) * 100
    # icd10 analysis
icd10 = map_df[
    (map_df['table_name'] == 'condition_occurrence') &
    (map_df['vocabulary_id'] == 'SNOMED') &
    (map_df['domain_id'] == 'Condition') &
    (map_df['source_vocabulary_id'] == 'ICD10GM')
]
total_icd10= icd10['record_count'].sum()
icd10_per_tumor_percentage = (total_icd10 / total_tumors) * 100

# [1c] diagnostic analysis
    # morphology
morph = map_df[
    (map_df['table_name'] == 'observation') &
    (map_df['concept_class_id'] == 'Morph Abnormality') &
     (map_df['domain_id'] == 'Observation') &
    (map_df['vocabulary_id'] == 'SNOMED')
]
total_morph = morph['record_count'].sum()
morph_per_tumor_percentage = (total_morph / total_tumors)
    # topography
topo = map_df[
    (map_df['table_name'] == 'observation') &
    (map_df['domain_id'] == 'Spec Anatomic Site') &
    (map_df['vocabulary_id'] == 'SNOMED')
]
total_topo = topo['record_count'].sum()
topo_per_tumor_percentage = (total_topo / total_tumors)


# [1d] vocabulary + concept analysis
total_concepts = map_df['record_count'].sum()
vocab_sum = map_df.groupby('vocabulary_id')['record_count'].sum().reset_index()
vocab_sum['percentage'] = (vocab_sum['record_count'] / total_concepts * 100).round(2)
vocab_sum = vocab_sum.sort_values(by='record_count', ascending=False)
domain_sum = map_df.groupby('domain_id')['record_count'].sum().reset_index()
domain_sum['percentage'] = (domain_sum['record_count'] / total_concepts * 100).round(2)
domain_sum = domain_sum.sort_values(by='record_count', ascending=False)

# printing results
print(f"Amount of mapped standardized concepts: {total_concepts}")
print(f"Amount of mapped ICD-O-3-entries in condition_occurrence: {total_icdo3}")
print(f"Amount of mapped ICD-10-entries in condition_occurrence: {total_icd10}")
print(f"Amount of mapped morphology-codes in observation: {total_morph}")
print(f"Amount of mapped topography-Codes in observation: {total_topo}")
print(f"Amount patiens: {total_patients}")
print(f"Amount tumors: {total_tumors}")
print(f"Average amount of ICD-O-3-entries per patient: {total_icdo3 / total_patients:.2f}")
print(f"ICD-O-3-mappings proportion relative to the tumor count: {icdo3_per_tumor_percentage:.2f}%")
print(f"Average amount of ICD-10-entries per patient: {total_icd10 / total_patients:.2f}")
print(f"ICD-10-mappings proportion relative to the tumor count: {icd10_per_tumor_percentage:.2f}%")
print(f"Average amount of morphology-entries per patient: {total_morph / total_patients:.2f}")
print(f"Morphology-mappings relative to the tumor count: {morph_per_tumor_percentage:.2f}%")
print(f"Average amount of topograpy-entries per patient: {total_topo / total_patients:.2f}")
print(f"Topography-Mappings relative to the tumor count: {topo_per_tumor_percentage:.2f}%")

# rendering results
display_table_with_caption(pivot_trans, 
                          "Amount of person which where successfully mapped and integrated into the target Common Data Model (CDM) system in comparison to the source system 'zfkd'.",
                          table_number=1)
display_table_with_caption(vocab_sum, 
                          "Amount of mapped concepts grouped by vocabulary.",
                          table_number=2)
display_table_with_caption(domain_sum, 
                          "Amount of mapped concepts grouped by domain.",
                          table_number=3)


In [ ]:
## [2a] episodic modelling analysis

# without hemonc
concept_class_counts = (
    episode[episode['concept_class_id'] != 'Regimen'] 
    .groupby('concept_class_id')['record_count']
    .sum()
    .reset_index()
)
concept_class_counts.columns = ['concept_class_id', 'concept_count']
total_episodes = concept_class_counts['concept_count'].sum()
concept_class_counts['percentage'] = (concept_class_counts['concept_count'] / total_episodes * 100).round(2)
concept_class_counts = concept_class_counts.sort_values(by='concept_count', ascending=False)
print(f"Number of all mapped concept belonging to the oncology extension (Episode table): {total_episodes}")


# treatment
treat = episode[
    (episode['concept_class_id'] == 'Treatment') 
]
treat_counts = treat.groupby('concept_name')['record_count'].sum().reset_index()
total_drugs = treat_counts.loc[treat_counts['concept_name'] == 'Cancer Drug Treatment', 'record_count'].sum()
treat_counts['percentage'] = (treat_counts['record_count'] / total_drugs * 100).round(2)
treat_counts

# HemOnc - not relevant right now
hemonc = episode[
    (episode['vocabulary_id'] == 'HemOnc') 
]

display_table_with_caption(concept_class_counts, 
                          "Amount of concept classes which where successfully mapped and integrated into the Episode Domain of the OMOP CDM",
                          table_number=4)
display_table_with_caption(treat_counts, 
                          "Amount of concepts which where successfully mapped and integrated into the Episode Domain grouped by treatment concept class of the OMOP CDM",
                          table_number=5)

# [2b] episode event analysis

# 1. count number of events per episode
events_per_episode = episode_event.groupby('episode_id').size().reset_index(name='event_count')

# 2. calculate average
average_events = events_per_episode['event_count'].mean()
print(f'Average {average_events:.2f} events per episode')

# 3. episodes with the most events (optional)
top_episodes = episode_event.groupby('concept_name').size().reset_index(name='event_count').sort_values(by='event_count', ascending=False)
#events_per_episode.sort_values(by='event_count', ascending=False).head(10)
print("Top 10 episodes:")
print(top_episodes)

# count number of events per episode_id and concept_name
events_per_episode_concept = (
    episode_event
    .groupby(['concept_name', 'episode_id'])
    .size()
    .reset_index(name='event_count')
)

summary = (
    events_per_episode_concept
    .groupby('concept_name')
    .agg(
        total_events = ('event_count', 'sum'),
        episode_count = ('episode_id', 'nunique')
    )
    .reset_index()
)

# calculate average events per episode
summary['avg_events_per_episode'] = summary['total_events'] / summary['episode_count']

# sort by average descending
summary_sorted = summary.sort_values(by='avg_events_per_episode', ascending=False)

print(summary_sorted[['concept_name', 'avg_events_per_episode']])
total_events = summary['total_events'].sum()
print(f'Total {total_events:.2f} events')
